In [2]:
from math import *


In [3]:
class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise(ValueError, "invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise(ValueError, "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise(ValueError, "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise(ValueError, "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise(ValueError, "Matrices must be m*n and n*p to multiply")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise(ValueError, "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise(ValueError, "Zero diagonal")
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)

In [4]:
class KalmanFilter():
    def __init__(self):
        x = ""
        P = ""
        F = ""
        H = ""
        R = ""
        I = ""
    
    def MeasurementUpdate(self, measurement):
        Z = matrix([measurement])
        #print("Z")
        #Z.show()
        y = Z.transpose() - (self.H * self.x)
        #print("y")
        #y.show()
        S = self.H * self.P * self.H.transpose() + self.R
        #print("S")
        #S.show()
        K = self.P * self.H.transpose() * S.inverse()
        #print("S_inv")
        #S.inverse().show()
        #print("K")
        #K.show()
        #print("H")
        #self.H.show()
        self.x = self.x + ( K * y )
        self.P = (self.I - (K * self.H)) * self.P
    
    def Prediction(self, externalMotion):
        self.x = (self.F * self.x) + externalMotion

        self.P = self.F * self.P * self.F.transpose()

    def GetState(self):
        return(self.x)
    
    def GetUncertainity(self):
        return(self.P)

In [12]:
kf = KalmanFilter()


In [13]:
dt = 0.1
kf.F = matrix([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])

In [14]:
kf.H = matrix([[1,0,0,0],[0,1,0,0]])
kf.R = matrix([[0.1,0],[0,0.1]])
kf.I = matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])

In [15]:
kf.x = matrix([[4],[12],[0],[0]])
kf.P = matrix([[10,0,0,0],[0,10,0,0],[0,0,1000,0],[0,0,0,1000]])

In [16]:
kf.x.show()
kf.P.show()
kf.F.show()
kf.H.show()
kf.R.show()
kf.I.show()

[4]
[12]
[0]
[0]
 
[10, 0, 0, 0]
[0, 10, 0, 0]
[0, 0, 1000, 0]
[0, 0, 0, 1000]
 
[1, 0, 0.1, 0]
[0, 1, 0, 0.1]
[0, 0, 1, 0]
[0, 0, 0, 1]
 
[1, 0, 0, 0]
[0, 1, 0, 0]
 
[0.1, 0]
[0, 0.1]
 
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]
 


In [17]:
measurements = [[5,10],[6,8],[7,6],[8,4],[9,2],[10,0]]

In [37]:
for i in range(len(measurements)):
    kf.Prediction(matrix([[0],[0],[0],[0]]))
    print("Prediction::state:")
    kf.x.show()
    print("P:")
    kf.P.show()
    
    kf.MeasurementUpdate(measurements[i])
    print("Measurement::state:")
    kf.x.show()
    print("P:")
    kf.P.show()
   

x
[4.0]
[12.0]
[0]
[0]
 
P
[20.0, 0.0, 100.0, 0.0]
[0.0, 20.0, 0.0, 100.0]
[100.0, 0.0, 1000, 0]
[0.0, 100.0, 0, 1000]
 
Prediction::state:
[4.0]
[12.0]
[0]
[0]
 
P:
[20.0, 0.0, 100.0, 0.0]
[0.0, 20.0, 0.0, 100.0]
[100.0, 0.0, 1000, 0]
[0.0, 100.0, 0, 1000]
 
Z
[5, 10]
 
y
[1.0]
[-2.0]
 
S
[20.1, 0.0]
[0.0, 20.1]
 
S_inv
[0.04975124378109452, -0.0]
[-0.0, 0.04975124378109452]
 
K
[0.9950248756218905, 0.0]
[0.0, 0.9950248756218905]
[4.975124378109452, 0.0]
[0.0, 4.975124378109452]
 
H
[1, 0, 0, 0]
[0, 1, 0, 0]
 
Measurement::state:
[4.9950248756218905]
[10.009950248756219]
[4.975124378109452]
[-9.950248756218905]
 
P:
[0.09950248756219082, 0.0, 0.4975124378109541, 0.0]
[0.0, 0.09950248756219082, 0.0, 0.4975124378109541]
[0.4975124378109541, 0.0, 502.48756218905476, 0.0]
[0.0, 0.4975124378109541, 0.0, 502.48756218905476]
 
x
[5.492537313432836]
[9.014925373134329]
[4.975124378109452]
[-9.950248756218905]
 
P
[5.223880597014929, 0.0, 50.74626865671643, 0.0]
[0.0, 5.223880597014929, 0.0, 5

In [19]:
kf.Prediction(matrix([[0],[0],[0],[0]]))
kf.MeasurementUpdate(measurements[0])

for i in range(100):
    kf.Prediction(matrix([[0],[0],[0],[0]]))
    print("X")
    kf.x.show()
    print("P")
    kf.P.show()
    kf.P.inverse().show()

X
[5.003219240549143]
[9.993561518901714]
[0.01811955280943689]
[-0.03623910561887378]
 
P
[0.1325143295463612, 0.0, 0.18300748337532122, 0.0]
[0.0, 0.1325143295463612, 0.0, 0.18300748337532122]
[0.18300748337531444, 0.0, 0.40467001274402037, 0.0]
[0.0, 0.18300748337531444, 0.0, 0.40467001274402037]
 
[20.10000000000355, -0.0, -9.09000000000361, -0.0]
[-0.0, 20.10000000000355, -0.0, -9.09000000000361]
[-9.09000000000361, -0.0, 6.582000000002936, -0.0]
[-0.0, -9.09000000000361, -0.0, 6.582000000002936]
 
X
[5.005031195830087]
[9.989937608339826]
[0.01811955280943689]
[-0.03623910561887378]
 
P
[0.17316252634886498, 0.0, 0.22347448464972325, 0.0]
[0.0, 0.17316252634886498, 0.0, 0.22347448464972325]
[0.22347448464971648, 0.0, 0.40467001274402037, 0.0]
[0.0, 0.22347448464971648, 0.0, 0.40467001274402037]
 
[20.100000000003828, -0.0, -11.100000000004119, -0.0]
[-0.0, 20.100000000003828, -0.0, -11.100000000004119]
[-11.100000000004119, -0.0, 8.60100000000378, -0.0]
[-0.0, -11.100000000004119